<a href="https://colab.research.google.com/github/DevMazaro/eBayPricePrediction/blob/main/eBay_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **eBay Laptop Sale Price Prediction**

##**Overview**
In the reverse logistics process, accurately predicting the potential resale value of a returned item is the critical first step in maximizing value recovery. This project develops a robust price prediction model for used laptops by leveraging a historical dataset of eBay sales from Kaggle.

After extensive data cleaning and feature engineering—especially standardizing the condition field—we will train and evaluate two distinct modeling approaches. We will compare a simple Linear Regression model as a baseline against a Multi-Layer Perceptron (MLP) Neural Network to capture more complex, non-linear relationships between a laptop's features and its final sale price.

The resulting price predictor is designed to be a key component in a larger decision engine, providing the crucial 'resale value' input needed to determine the most profitable disposition path for a returned laptop: resell as-is, refurbish and sell, liquidate, or recycle.

In [1]:
#Importing libraries to be used in the project

#Colab drive is used only during development
#from google.colab import drive

#Data manipulation and analysis
import pandas as pd
import numpy as np



In [2]:
#Read raw data file
#Reading from mounted google colab drive during development


df = pd.read_csv('/content/EbayPcLaptopsAndNetbooksRaw.csv')

#TODO: Read data file from github

#Data exploration and analysis
#Basic Dataset info and sample data
print("\n---Data Exploration and analysis\n")
print(f"Dataset shape:{df.shape}")
print(f"Dataset columns:\n {df.columns}")
print("First 10 rows:\n")
with pd.option_context('display.max_columns', None):
  print(df.head(10))

#Check for missing values
print(f"\nMissing Values:\n{df.isnull().sum()}")

#Statistical summary of data
print(f"\nStatistical Summary:\n{df.describe}")


---Data Exploration and analysis

Dataset shape:(6620, 23)
Dataset columns:
 Index(['Brand', 'Price', 'Rating', 'Ratings Count', 'Condition', 'Seller Note',
       'Processor', 'Screen Size', 'Manufacturer Color', 'Color', 'Ram Size',
       'SSD Capacity', 'GPU', 'Processor Speed', 'Type', 'Release Year',
       'Maximum Resolution', 'Model', 'OS', 'Features', 'Hard Drive Capacity',
       'Country Region Of Manufacture', 'Storage Type'],
      dtype='object')
First 10 rows:

   Brand               Price              Rating  Ratings Count  \
0  CHUWI            $303.68                  NaN            NaN   
1   Dell  $399.99 to $634.99                 NaN            NaN   
2   Dell            $175.00                  NaN            NaN   
3     HP             $84.99                  NaN            NaN   
4   Dell            $101.22                  NaN            NaN   
5   Acer             $49.99   4.5 out of 5 stars          193.0   
6     HP             $39.59   4.5 out of 5 stars

##**Initial Data Challenges**
As observed in the basic data exploration, we have many data challenges to be overcome. Including:

* The Price (Target Variable) is not numeric and contains diverse formats, including ranges.
* Missing Brand data (39% of records), which is a big factor for pricing.
* Computer specs in different formats.

Proposed approach for these issues:
* Clean the data and engineer features that can be used when training our models. Such as numeric price, or ordinal categories for computer specs.
* Iteratively engineer features we deem more important, train our models, and decide which features to use based on observed results.
   * Ex: Should we remove the datapoints that have no brand or should we create an unknown brand category?
   * Ex: Extract processor tier (i7 vs i5) vs parsing detailed processor specifications?

###**Data Clean-UP: Price (Target Variable)**
The initial analysis indicated two patterns:
*   Single price("\$X,XXX.XX")
*   Price range("\$X,XXX.XX to \$X,XXX.XX")
   *   For the purposes of this project, we decided to use the price range average as the new price.

We will cleanup these patterns and analyze the field again to check if more treatment is needed.

In [9]:
#The initial analysis of the analysis indicates two patterns:

#Function to clan the price column and transfort into a float.
def clean_price(price_str):
  price_str = price_str.strip()
  if ' to 'in price_str:
    prices = price_str.split(' to ')
    price1 = float(prices[0].replace('$','').replace(',',''))
    price2 = float(prices[1].replace('$','').replace(',',''))
    return round((price1+price2)/2,2)
  else:
    return round(float(price_str.replace('$','').replace(',','')),2)

#Aplly cleaning function
df['price_clean'] = df['Price'].apply(clean_price)

#Get the statistical summary of the clean price
print(f"Statistical Summary of new price:\n {df['price_clean'].describe()}")

#Random sample
print(f"\nRandom Sample of 5 rows:\n{df.sample(5, random_state=42)}")

#Taking a look at the row with the highest price
print (f"Row with highest price:\n{df.loc[df['price_clean'].idxmax()]}")

#It seems we don't need any more treatment in this field.

Statistical Summary of new price:
 count    6620.000000
mean      310.046627
std       399.076572
min         4.730000
25%        90.990000
50%       193.120000
75%       387.137500
max      3901.000000
Name: price_clean, dtype: float64

Random Sample of 5 rows:
     Brand     Price Rating  Ratings Count  \
96     NaN   $59.42     NaN            NaN   
994    NaN  $130.02     NaN            NaN   
1400   NaN   $70.60     NaN            NaN   
865    NaN  $338.76     NaN            NaN   
6095  Dell  $105.01     NaN            NaN   

                                              Condition  \
96    New: A brand-new, unused, unopened, undamaged ...   
994   New: A brand-new, unused, unopened, undamaged ...   
1400  New: A brand-new, unused, unopened, undamaged ...   
865   Used: An item that has been used previously. T...   
6095  UsedAn item that has been used previously. The...   

                                            Seller Note  \
96                                            

###**Data Clean-up: Brand**
The initial analysis for brand (see below) showed us that this field needs much more cleaning than Price. Including:
* A number of words and phrases that should be considered "unknown" (Ex. "Major Brand" and "?)".
* Records with multiple unrelated brands
* Part Numbers as Brand
* Specific Models as Brand
* Other entries as Brand (ex. "LCD Replacement")

In [30]:
# Check unique brand values
# Null values dropped to allow sorting
sorted_brands = sorted(df['Brand'].dropna().unique())

print(f"Numer of unique brands: {len(sorted_brands)}")
for brand in sorted_brands:
   print(brand)

print(f"\nUnique values separated by a slash:\n{df[df['Brand'].str.contains('/', na=False)]['Brand'].unique()}")

Numer of unique brands: 134
?
ACER
AIRBAR
AMIAMO
ASUS
AU Optronics
AUO
AUUSDA
Acer
Acer / IOmega
Alienware
Apple
Apple / LG
Asem
Asus
Bormann
C600
CHUNGHWA
CHUNGWA
CHUWI
CLAA116WA03A
CLAA156WA11A
CLAA156WA13A
CLEVO
Casio
ChiMei
ChiMei INNOLUX
Chimei INNOLUX
Chingwa
Chunghwa
Chuwi
Compaq
Craig
DELL
DELL/HP/LENOVO/Fujitsu/Toshiba
Dell
Dell 16gb
Dell Inc.
Dell Inspiron
Dell Latitude
Dell Latitude 7275
Dell/HP/Lenovo/Stone
Delta
Delta / DELL
Different
Does not apply
Durabook
FEC
Frontier
Fuji
Fujitsu
Fujitsu PC Corp.
Fujitsu Siemens
Fusion5
GERICOM
GETAC
GIGABYTE
GPD
Gateway
Geo
Getac
GoldBook
Google
HP
HP DELL LENOVO SELECT FROM ABOVE
HW13WX001
HannStar
HannnStar
HiDes
Huawei
Hyundai
IBM
INNOLUX
IVO
InnoLux
Insignia
Intel
JVC
Jumper
KODAK
Kano
Kodak
LCD Replacement
LENOVO
LG
Lenova
Lenovo
Lenovo T440
Lenovo T530
LiteON
MEDION
MSI
Major Brand
Media Wave
Metabox
Microsoft
Mixt
Motion Computing
Mouse Computer
Not Applicable
Olidata
Olpc
Olympus
Orbitkey
PANASONIC
Packard Bell
Packard bell
Pa

**Proposed Clean-up Logic**


1. Null Brands are considered "unknown"
2. The case for the Brand will be standardized before comparison.
3. Brands in the bad brands list are considered "unknown"
4. Brands with forward slash ("/") are considered unknown.
  * The number of affected records in this dataset is small.
  * This rule is justified in a real-world scenario as the "/" indicates uncertainty about the Brand or a mixed item. Still, we would revisit this rule with a different datase.
  * We are applying this generic rule after the more specific bad brands match.
5. We will match the brands with a list of "known brands".
   * For this project, we sourced the list from wikipedia (https://en.wikipedia.org/wiki/List_of_laptop_brands_and_manufacturers)
6. Finally, if there are no matches, we will consider the brand as "unknown"


